# CIFAR Dataset

> Load the whole/part CIFAR dataset

In [ ]:
# | default_exp data.load_cifar

In [ ]:
# | hide
from nbdev.showdoc import *

%reload_ext jupyternotify
%reload_ext nb_black
%reload_ext autoreload
%reload_ext line_profiler
%autoreload 2
%env CUDA_VISIBLE_DEVICES=

import sys

__root = "../../"
sys.path.append(__root)

In [ ]:
# | export
from torch_snippets import *
from simple_ddpm.config.cifar import Config

In [ ]:
# | export
def denorm(img):
    if isinstance(img, list) or img.ndim >= 4:
        return torch.cat([denorm(_img)[None] for _img in img])
    img = img - img.min()
    img = img / img.max()
    img = img.permute(1, 2, 0)
    return img.float()


class CIFAR:
    """Load CIFAR10 cars dataset"""

    def __init__(self, root, config, **kwargs):
        store_attr()
        self.transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
            ]
        )
        self.reverse_transform = transforms.Compose(
            [
                transforms.Lambda(denorm),
            ]
        )
        if kwargs.pop("load_on_init", True):
            self.data = self.load_trainset()

    def __getitem__(self, ix):
        return self.data[ix]

    def load_trainset(
        self,
        train: bool = True,  # Is it train or validation subset
        download: bool = True,
    ):  # Shoud I download if it doesn't exist?
        trainset = torchvision.datasets.CIFAR10(
            root=self.root, train=train, download=download, transform=self.transform
        )
        return trainset

    def make_dataloader(self, collate_function=None):
        dataloader = torch.utils.data.DataLoader(
            self.data,
            batch_size=self.config.BATCH_SIZE,
            shuffle=True,
            num_workers=0,
            collate_fn=collate_function,
        )
        return dataloader

    def filter_by_label(
        self, trainset: Dataset, label_id: int  # CIFAR Dataset
    ):  # Which label id do you want to filter (e.g., 1 is Car)
        cache = f"{self.root}/{label_id}.idxs"
        if exists(cache):
            idx = loaddill(cache)
        else:
            with notify_waiting(f"Filtering cifar for label-id {label_id}"):
                idx = [
                    i for i, (img, label) in enumerate(trainset) if label == label_id
                ]
            dumpdill(idx, cache)
        sub_trainset = torch.utils.data.Subset(trainset, idx)
        return sub_trainset

    def __len__(self):
        return len(self.data)

    def batch(self, ixs=None, n=9):
        if ixs is None:
            ixs = [randint(len(self)) for _ in range(n)]
        ims = torch.cat([self[ix][0][None] for ix in ixs])
        return ims

    def show_samples(self, ixs=None, n=9, **kws):
        if ixs is not None:
            n = len(ixs)
        ims = self.batch(ixs=ixs, n=n)
        self.show_images(ims, **kws)

    def show_images(self, ims, **kws):
        ims = [self.reverse_transform(im) for im in ims]
        n = len(ims)
        if "nc" not in kws:
            nc = np.sqrt(n)
            nc = int(nc) + 1 if int(nc) != nc else int(nc)
        else:
            nc = kws.pop("nc")
        subplots(ims, nc=nc, **kws) if n > 1 else show(ims[0], **kws)
        plt.tight_layout()

    def generate_noise(self, images):
        t = np.random.randint(0, self.config.timesteps, size=len(images))
        a = self.config.time_bar[t].reshape(-1, 1, 1, 1)
        b = self.config.time_bar[t + 1].reshape(-1, 1, 1, 1)
        noise = np.random.normal(size=images.shape)
        img_a = (images * (1 - a) + noise * a).float()
        img_b = (images * (1 - b) + noise * b).float()
        return {"t": torch.Tensor(t).float()[..., None], "x": img_a, "y": img_b}

    def generate_noise_collator(self, batch):
        ims, _ = zip(*batch)
        ims = torch.stack(ims)
        return self.generate_noise(ims)


class CIFARCars(CIFAR):
    def __init__(self, root, config):
        super().__init__(root, config, load_on_init=False)
        self.data = self.get_cifar_cars()

    def get_cifar_cars(self):
        return self.filter_by_label(self.load_trainset(self.root), label_id=1)

In [ ]:
config = Config()
config.BATCH_SIZE = 4
data = CIFAR("/Users/yeshwanth.y/Downloads/data", config)
ims, _ = next(iter(data.make_dataloader()))
data.show_images(ims, nc=len(ims))
batch = data.generate_noise(ims)
a, b = batch["x"], batch["y"]
subplots(
    [*denorm(a), *denorm(b)],
    nc=len(a),
    titles=flatten([["input"] * len(a), ["output"] * len(b)]),
)

In [ ]:
config = Config()
config.BATCH_SIZE = 4
data = CIFAR("/Users/yeshwanth.y/Downloads/data", config)
dl = data.make_dataloader(collate_function=data.generate_noise_collator)
batch = next(iter(dl))
a, b = batch["x"], batch["y"]
subplots(
    [*denorm(a), *denorm(b)],
    nc=len(a),
    titles=flatten([["input"] * len(a), ["output"] * len(b)]),
)

In [ ]:
config = Config()
config.BATCH_SIZE = 8
data = CIFARCars("/Users/yeshwanth.y/Downloads/data", config)
dl = data.make_dataloader(collate_function=data.generate_noise_collator)
batch = next(iter(dl))
a, b = batch["x"], batch["y"]
subplots(
    [*denorm(a), *denorm(b)],
    nc=len(a),
    titles=flatten([["input"] * len(a), ["output"] * len(b)]),
)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()